In [49]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import string
import numpy as np
import pickle

In [50]:
# Load model
from keras.models import load_model

model=load_model("./Model/Model_Chatbot_Personal.h5")
import json
import random

with open("./Dataset/intents.json",encoding="utf-8") as file:
    intents=json.load(file)
with open("./Dataset/words.pkl","rb") as file:
    words=pickle.load(file)
with open("./Dataset/classes.pkl","rb") as file:
    classes=pickle.load(file)
    
# print(open("./Dataset/intents.json").read())
# intents2=json.loads(open("./Dataset/intents.json",encoding="utf-8").read())

In [51]:
# print(intents)
# print(intents2)

In [52]:
print(words)
print(classes)

['hi', 'anyone', 'hey', 'hola', 'hello', 'good', 'day', 'bye', 'see', 'later', 'goodbye', 'nice', 'chat', 'bye', 'till', 'next', 'time', 'thanks', 'thank', "'s", 'helpful', 'awesome', 'thanks', 'thanks', 'help', 'could', 'help', 'help', 'provide', 'helpful', 'support', 'offer', 'check', 'adverse', 'drug', 'reaction', 'open', 'adverse', 'drug', 'module', 'give', 'list', 'drug', 'cause', 'adverse', 'behavior', 'list', 'drug', 'suitable', 'patient', 'adverse', 'reaction', 'drug', 'dont', 'adverse', 'reaction', 'open', 'blood', 'pressure', 'module', 'task', 'relate', 'blood', 'pressure', 'blood', 'pressure', 'data', 'entry', 'want', 'log', 'blood', 'pressure', 'result', 'blood', 'pressure', 'data', 'management', 'want', 'search', 'blood', 'pressure', 'result', 'history', 'blood', 'pressure', 'patient', 'load', 'patient', 'blood', 'pressure', 'result', 'show', 'blood', 'pressure', 'result', 'patient', 'find', 'blood', 'pressure', 'result', 'id', 'find', 'pharmacy', 'find', 'pharmacy', 'list

In [53]:
lemmatizer=WordNetLemmatizer()
punc=string.punctuation
stop_words=stopwords.words("english")

# function to clean
def remove_punctuation(w_list):
    return [word for word in w_list if word not in punc]

def remove_stopwords(w_list):
    return [word for word in w_list if word not in stop_words]

def get_tag(tag):
    if tag.startswith('j'):
        return 'a'
    elif tag.startswith('v'):
        return 'v'
    elif tag.startswith('n'):
        return 'n'
    elif tag.startswith('r'):
        return 'r'
    else:
        return None

def lemmatize(w_list):
    lemmatized=[]
    tagging=pos_tag(w_list)
    for word,tag in tagging:
        tag=get_tag(tag.lower())
        if tag is None:
            lemmatized.append(word.lower())
        else:
            lemmatized.append(lemmatizer.lemmatize(word.lower(),tag))
    return lemmatized

In [54]:
def clean_up_sentence(msg):
    sentence_words=word_tokenize(msg)
    # print("Sentence word:",sentence_words)
    sentence_words=remove_punctuation(sentence_words)
    sentence_words=remove_stopwords(sentence_words)
    sentence_words=lemmatize(sentence_words)
    return sentence_words

In [55]:
def bag_of_words(msg,words,show_details=True):
    # tokenize
    msg=clean_up_sentence(msg)
    # print("Cleaned up sentence:",msg)
    
    # bag of words - matrix of N words
    bag = [0]*len(words)
    for s in msg:
        for i,w in enumerate(words):
            if w==s:
                # assign 1 if current word is in the vocabulary (words)
                bag[i]=1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

In [56]:
def predict_class(msg,model):
    # filter below threshold
    p=bag_of_words(msg,words,show_details=False)
    # print("p:",p)
    res=model.predict(np.array([p]),verbose=0)[0]
    # print("res:",res)
    err_threshold=0.05
    
    result=[[i,r] for i,r in enumerate(res) if r>err_threshold]
    if len(result)==0:
        print("I dont know what u mean!")
        return -1
    # print("Result:",result)
    
    # sorting
    result.sort(key=lambda x:x[1],reverse=True)
    # print("Result:",result)
    
    result_list=[]
    # for r in result:
    result_list.append({"intent":classes[result[0][0]],"probability":str(result[0][1])})
    return result_list
    

In [57]:
def get_response(int_pred,intents_json):
    tag=int_pred[0]["intent"]
    # print("Tag:",tag)
    list_of_intents=intents_json["intents"]
    if tag in ["goodbye","thanks"]:
        for i in list_of_intents:
            if i["tag"]==tag:
                result=random.choice(i["responses"])
                break
        print("Bot:",result)
        return -1
    for i in list_of_intents:
        if i["tag"].strip()==tag.strip():
            result=random.choice(i["responses"])
            break
    return result

In [58]:
def chatbot_response(msg):
    int_predict=predict_class(msg,model)
    if int_predict == -1:
        return int_predict
    # print("Int predict:",int_predict)
    response=get_response(int_predict,intents)
    # if response==-1:
    #     return 0
    # print("Chatbot response:",response)
    return response

In [47]:
print(f'Chatbot response: {chatbot_response("what is deep learning?")}')

Sentence word: ['what', 'is', 'deep', 'learning', '?']
Cleaned up sentence: ['deep', 'learning']
p: [0 0 0 ... 0 0 0]
res: [2.41610874e-08 1.48378561e-07 2.34865905e-10 2.02018370e-07
 9.59115912e-11 3.44722615e-08 3.19570724e-11 2.06977413e-09
 1.11779941e-09 2.35055059e-10 1.35972567e-09 2.84921444e-11
 2.77945378e-09 1.11661691e-09 1.45798193e-10 1.53206378e-10
 6.86800172e-10 2.13265072e-09 6.72238931e-11 3.40665829e-10
 9.54773128e-01 2.41686116e-11 1.36186245e-08 6.51611320e-10
 3.63049524e-09 6.83336907e-08 1.11128752e-10 1.66891656e-08
 2.19898055e-09 3.62120067e-09 2.07514206e-09 5.50492762e-10
 8.15635837e-10 3.43207407e-10 1.17173602e-06 1.03407267e-07
 7.14681037e-10 1.65406244e-09 5.09403006e-11 2.75949975e-11
 7.69320962e-09 6.99675817e-10 2.34731363e-08 2.48442063e-07
 1.72969798e-12 1.64100360e-08 4.79748508e-09 1.74684811e-09
 9.08512710e-11 3.18635870e-07 5.61453835e-11 1.31084548e-07
 2.55246685e-10 2.66968565e-11 2.41228605e-07 2.75987233e-08
 2.00220882e-10 4.80311

In [59]:
# buatkan templatge pembicaraan dengan chatbotnya di sini
while True:
    msg=input("You: ")
    response=chatbot_response(msg)
    if response==-1:
        break
    print("Bot:",response.strip())
    # if response[0]=="goodbye" or response[0]=="thanks":
    #     break

Bot: Hi there, how can I help?
Bot: I can answer your questions related to machine learning or any general anser. Come on try me
Bot: Reinforcement Learning(RL) is a type of machine learning technique that enables an agent to learn in an interactive environment by trial and error using feedback from its own actions and experiences.
Bot: Any time!


: 

#### Flask

In [36]:
# from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify

app=Flask(__name__)

# run_with_ngrok(app)

@app.route("/",methods=["GET"])
def hello():
    return jsonify({"message":"Running successfully"})

def decrypt(msg):
    # input : what+is+deep+learning
    # output : what is deep learning
    # remove + , replace with space
    
    string=msg
    new_string=string.replace("+"," ")
    
    return new_string

@app.route("/query/<sentence>")
def chatbot_query(sentence):
    
    # decrypt
    dec_msg=decrypt(sentence)
    
    response=chatbot_response(dec_msg)
    
    json_response=jsonify({"top" : {"response":response}})

    return json_response

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [06/Apr/2024 02:46:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2024 02:46:45] "GET /query/what+is+deep+learning HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2024 02:47:50] "GET /query/hi+there HTTP/1.1" 200 -


In [1]:
!pip list

Package                   Version
------------------------- ----------
anyio                     4.1.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.1
beautifulsoup4            4.12.2
bleach                    6.1.0
blinker                   1.7.0
certifi                   2021.5.30
cffi                      1.16.0
chardet                   4.0.0
charset-normalizer        3.3.2
click                     8.1.7
colorama                  0.4.6
comm                      0.2.0
contourpy                 1.2.0
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
distlib                   0.3.7
executing                 2.0.1
fastjsonschema            2.19.0
filelock                  3.13.1
Flask                     3.0.2
flask-ngrok         


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\micha\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip
